In [1]:
import os
import csv
import math
import wandb
import numpy as np
from pathlib import Path 

from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from wandb.keras import WandbCallback
from keras.layers import Dense, Flatten, Dropout

Using TensorFlow backend.


In [2]:
#Getting the latest file from the folder
record_path = Path("RecordSummary")
training_data_list = list(filter(os.path.isfile,record_path.glob('*.csv')))
training_data_list.sort(key=lambda x: os.path.getmtime(x))
training_data = training_data_list[-1]

x_train_percent = 60
x_test_percent = 100-x_train_percent
x_train = []
x_test = []
y_train = []
y_test = []

with open(training_data) as csv_file:
    data = list(csv.reader(csv_file))
    lines = list(map(list, zip(*data)))

nr_x_train = math.ceil(len(lines[0])*x_train_percent/100)
lines = np.asarray(lines)


#X stands for input, Y stands for output

x_train = lines[1:, 0:nr_x_train]
y_train = lines[0, 0:nr_x_train]

x_test = lines[1:, nr_x_train:] 
y_test = lines[0, nr_x_train:]
x_train = x_train.transpose()
x_test = x_test.transpose()

x_train = [a.reshape(4,96) for a in x_train]
x_test = [a.reshape(4,96) for a in x_test]

is_up_y_train = y_train=='up'
is_up_y_test = y_test=='up'
labels = ["Not UP", "UP"]

In [4]:
channels = np.shape(x_train)[1]
samples = np.shape(x_train)[2]

print(channels)
print(samples)

4
96


In [5]:
print(np.shape(x_train))
print(np.shape(is_up_y_train))
print(np.shape(x_test))
print(np.shape(is_up_y_test))

(132, 4, 96)
(132,)
(87, 4, 96)
(87,)


In [6]:
print(type(x_train))
print(type(is_up_y_train))

<class 'list'>
<class 'numpy.ndarray'>


In [7]:
#converting <class 'list'> to <class 'numpy.ndarray'>
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

In [8]:
# create model
model=Sequential()
model.add(Flatten(input_shape=(channels,samples)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam',
                metrics=['accuracy'])

In [13]:
# logging code
run = wandb.init(project="eeg-up-down")
config = run.config
config.epochs = 1000
# Fit the model
model.fit(x_train, is_up_y_train, epochs=config.epochs, validation_data=(x_test, is_up_y_test),
                    callbacks=[WandbCallback(labels=labels)])

Train on 132 samples, validate on 87 samples
Epoch 1/1000
132/132 [==============================] - 0s 392us/step - loss: 61.3210 - accuracy: 0.0455 - val_loss: 948.1971 - val_accuracy: 0.0345
Epoch 2/1000
132/132 [==============================] - 0s 318us/step - loss: 57.7461 - accuracy: 0.0455 - val_loss: 940.5435 - val_accuracy: 0.0230
Epoch 3/1000
132/132 [==============================] - 0s 296us/step - loss: 59.7476 - accuracy: 0.0682 - val_loss: 905.4281 - val_accuracy: 0.0230
Epoch 4/1000
132/132 [==============================] - 0s 311us/step - loss: 55.9734 - accuracy: 0.0455 - val_loss: 866.8556 - val_accuracy: 0.0345
Epoch 5/1000
132/132 [==============================] - 0s 311us/step - loss: 53.2499 - accuracy: 0.0833 - val_loss: 769.9733 - val_accuracy: 0.0345
Epoch 6/1000
132/132 [==============================] - 0s 303us/step - loss: 62.7524 - accuracy: 0.0379 - val_loss: 767.8832 - val_accuracy: 0.0230
Epoch 7/1000
132/132 [==============================] - 0s 31

132/132 [==============================] - 0s 295us/step - loss: 28.1634 - accuracy: 0.0758 - val_loss: 603.9291 - val_accuracy: 0.0920
Epoch 56/1000
132/132 [==============================] - 0s 310us/step - loss: 26.2816 - accuracy: 0.0833 - val_loss: 612.7644 - val_accuracy: 0.1034
Epoch 57/1000
132/132 [==============================] - 0s 310us/step - loss: 25.5529 - accuracy: 0.0455 - val_loss: 608.6323 - val_accuracy: 0.0345
Epoch 58/1000
132/132 [==============================] - 0s 295us/step - loss: 26.5237 - accuracy: 0.0606 - val_loss: 564.4742 - val_accuracy: 0.0920
Epoch 59/1000
132/132 [==============================] - 0s 303us/step - loss: 24.8302 - accuracy: 0.0606 - val_loss: 525.8172 - val_accuracy: 0.0805
Epoch 60/1000
132/132 [==============================] - 0s 311us/step - loss: 25.0000 - accuracy: 0.0606 - val_loss: 485.6211 - val_accuracy: 0.0345
Epoch 61/1000
132/132 [==============================] - 0s 289us/step - loss: 27.7506 - accuracy: 0.0606 - val_lo

132/132 [==============================] - 0s 296us/step - loss: 22.7434 - accuracy: 0.0530 - val_loss: 396.0012 - val_accuracy: 0.0345
Epoch 110/1000
132/132 [==============================] - 0s 303us/step - loss: 18.1924 - accuracy: 0.0833 - val_loss: 373.2666 - val_accuracy: 0.0230
Epoch 111/1000
132/132 [==============================] - 0s 340us/step - loss: 25.1167 - accuracy: 0.1288 - val_loss: 423.4191 - val_accuracy: 0.0345
Epoch 112/1000
132/132 [==============================] - 0s 288us/step - loss: 29.8993 - accuracy: 0.0758 - val_loss: 365.3358 - val_accuracy: 0.0460
Epoch 113/1000
132/132 [==============================] - 0s 296us/step - loss: 41.1961 - accuracy: 0.0606 - val_loss: 432.6356 - val_accuracy: 0.0230
Epoch 114/1000
132/132 [==============================] - 0s 296us/step - loss: 37.5305 - accuracy: 0.0682 - val_loss: 360.4758 - val_accuracy: 0.0575
Epoch 115/1000
132/132 [==============================] - 0s 303us/step - loss: 34.0378 - accuracy: 0.0985 - 

132/132 [==============================] - 0s 303us/step - loss: 15.1581 - accuracy: 0.0758 - val_loss: 301.1798 - val_accuracy: 0.0460
Epoch 164/1000
132/132 [==============================] - 0s 303us/step - loss: 15.5252 - accuracy: 0.0985 - val_loss: 312.0880 - val_accuracy: 0.0345
Epoch 165/1000
132/132 [==============================] - 0s 296us/step - loss: 12.6105 - accuracy: 0.1212 - val_loss: 303.0630 - val_accuracy: 0.0575
Epoch 166/1000
132/132 [==============================] - 0s 296us/step - loss: 13.5958 - accuracy: 0.0682 - val_loss: 304.4355 - val_accuracy: 0.0575
Epoch 167/1000
132/132 [==============================] - 0s 303us/step - loss: 12.4662 - accuracy: 0.0833 - val_loss: 297.2334 - val_accuracy: 0.0575
Epoch 168/1000
132/132 [==============================] - 0s 335us/step - loss: 12.0415 - accuracy: 0.0909 - val_loss: 285.6644 - val_accuracy: 0.0345
Epoch 169/1000
132/132 [==============================] - 0s 296us/step - loss: 13.0051 - accuracy: 0.0606 - 

132/132 [==============================] - 0s 296us/step - loss: 11.8416 - accuracy: 0.0833 - val_loss: 259.1211 - val_accuracy: 0.0805
Epoch 218/1000
132/132 [==============================] - 0s 303us/step - loss: 12.7377 - accuracy: 0.0909 - val_loss: 248.7509 - val_accuracy: 0.0575
Epoch 219/1000
132/132 [==============================] - 0s 303us/step - loss: 8.8154 - accuracy: 0.1288 - val_loss: 259.4262 - val_accuracy: 0.0575
Epoch 220/1000
132/132 [==============================] - 0s 303us/step - loss: 10.8093 - accuracy: 0.1515 - val_loss: 262.0286 - val_accuracy: 0.0690
Epoch 221/1000
132/132 [==============================] - 0s 303us/step - loss: 8.8180 - accuracy: 0.1591 - val_loss: 266.5799 - val_accuracy: 0.0805
Epoch 222/1000
132/132 [==============================] - 0s 296us/step - loss: 8.5200 - accuracy: 0.1742 - val_loss: 277.7540 - val_accuracy: 0.0345
Epoch 223/1000
132/132 [==============================] - 0s 296us/step - loss: 9.4785 - accuracy: 0.1667 - val_

...

Epoch 761/1000
132/132 [==============================] - 0s 296us/step - loss: 0.9031 - accuracy: 0.3939 - val_loss: 88.3447 - val_accuracy: 0.0690
Epoch 762/1000
132/132 [==============================] - 0s 303us/step - loss: 0.9925 - accuracy: 0.3561 - val_loss: 85.0423 - val_accuracy: 0.0805
Epoch 763/1000
132/132 [==============================] - 0s 303us/step - loss: 0.9196 - accuracy: 0.3939 - val_loss: 82.7977 - val_accuracy: 0.0345
Epoch 764/1000
132/132 [==============================] - 0s 311us/step - loss: 2.5664 - accuracy: 0.3636 - val_loss: 88.3926 - val_accuracy: 0.0345
Epoch 765/1000
132/132 [==============================] - 0s 348us/step - loss: 2.0093 - accuracy: 0.3030 - val_loss: 96.1463 - val_accuracy: 0.0230
Epoch 766/1000
132/132 [==============================] - 0s 318us/step - loss: 8.2114 - accuracy: 0.0909 - val_loss: 79.0096 - val_accuracy: 0.0690
Epoch 767/1000
132/132 [==============================] - 0s 304us/step - loss: 6.6372 - accuracy: 0.1288 

Epoch 816/1000
132/132 [==============================] - 0s 318us/step - loss: 1.0058 - accuracy: 0.3636 - val_loss: 84.6303 - val_accuracy: 0.0230
Epoch 817/1000
132/132 [==============================] - 0s 318us/step - loss: 1.3551 - accuracy: 0.2273 - val_loss: 85.3063 - val_accuracy: 0.0690
Epoch 818/1000
132/132 [==============================] - 0s 315us/step - loss: 1.7822 - accuracy: 0.2273 - val_loss: 84.3865 - val_accuracy: 0.0345
Epoch 819/1000
132/132 [==============================] - 0s 303us/step - loss: 0.8189 - accuracy: 0.4318 - val_loss: 86.2657 - val_accuracy: 0.0115
Epoch 820/1000
132/132 [==============================] - 0s 318us/step - loss: 0.6240 - accuracy: 0.4318 - val_loss: 86.5826 - val_accuracy: 0.0460
Epoch 821/1000
132/132 [==============================] - 0s 318us/step - loss: 0.6682 - accuracy: 0.4924 - val_loss: 86.5968 - val_accuracy: 0.0805
Epoch 822/1000
132/132 [==============================] - 0s 421us/step - loss: 0.8327 - accuracy: 0.3939 

Epoch 871/1000
132/132 [==============================] - 0s 311us/step - loss: 1.0359 - accuracy: 0.3182 - val_loss: 81.6813 - val_accuracy: 0.0345
Epoch 872/1000
132/132 [==============================] - 0s 310us/step - loss: 1.7912 - accuracy: 0.2424 - val_loss: 79.7580 - val_accuracy: 0.0230
Epoch 873/1000
132/132 [==============================] - 0s 318us/step - loss: 0.7771 - accuracy: 0.4091 - val_loss: 79.6078 - val_accuracy: 0.0460
Epoch 874/1000
132/132 [==============================] - 0s 362us/step - loss: 0.5102 - accuracy: 0.5606 - val_loss: 80.1140 - val_accuracy: 0.0575
Epoch 875/1000
132/132 [==============================] - 0s 303us/step - loss: 0.4498 - accuracy: 0.6061 - val_loss: 81.4467 - val_accuracy: 0.0460
Epoch 876/1000
132/132 [==============================] - 0s 296us/step - loss: 0.4632 - accuracy: 0.5152 - val_loss: 81.8527 - val_accuracy: 0.0460
Epoch 877/1000
132/132 [==============================] - 0s 303us/step - loss: 0.4293 - accuracy: 0.5606 

Epoch 926/1000
132/132 [==============================] - 0s 296us/step - loss: 1.1258 - accuracy: 0.3333 - val_loss: 84.2329 - val_accuracy: 0.1034
Epoch 927/1000
132/132 [==============================] - 0s 288us/step - loss: 1.1970 - accuracy: 0.2955 - val_loss: 77.7046 - val_accuracy: 0.0920
Epoch 928/1000
132/132 [==============================] - 0s 296us/step - loss: 0.8996 - accuracy: 0.4167 - val_loss: 82.1075 - val_accuracy: 0.1034
Epoch 929/1000
132/132 [==============================] - 0s 311us/step - loss: 0.9871 - accuracy: 0.4091 - val_loss: 87.9064 - val_accuracy: 0.0920
Epoch 930/1000
132/132 [==============================] - 0s 295us/step - loss: 0.7978 - accuracy: 0.3864 - val_loss: 87.7303 - val_accuracy: 0.0920
Epoch 931/1000
132/132 [==============================] - 0s 319us/step - loss: 0.6783 - accuracy: 0.4394 - val_loss: 86.5064 - val_accuracy: 0.0460
Epoch 932/1000
132/132 [==============================] - 0s 318us/step - loss: 0.4035 - accuracy: 0.5909 

Epoch 981/1000
132/132 [==============================] - 0s 303us/step - loss: 1.2100 - accuracy: 0.5758 - val_loss: 93.9897 - val_accuracy: 0.1149
Epoch 982/1000
132/132 [==============================] - 0s 296us/step - loss: 2.5674 - accuracy: 0.3258 - val_loss: 88.2103 - val_accuracy: 0.1034
Epoch 983/1000
132/132 [==============================] - 0s 303us/step - loss: 1.4829 - accuracy: 0.3561 - val_loss: 78.4452 - val_accuracy: 0.1149
Epoch 984/1000
132/132 [==============================] - 0s 303us/step - loss: 1.8112 - accuracy: 0.2273 - val_loss: 78.8110 - val_accuracy: 0.0230
Epoch 985/1000
132/132 [==============================] - 0s 295us/step - loss: 1.9072 - accuracy: 0.2348 - val_loss: 81.6065 - val_accuracy: 0.0690
Epoch 986/1000
132/132 [==============================] - 0s 326us/step - loss: 1.6693 - accuracy: 0.2727 - val_loss: 76.7300 - val_accuracy: 0.1034
Epoch 987/1000
132/132 [==============================] - 0s 303us/step - loss: 0.8912 - accuracy: 0.3864 